In [1]:
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns
from lxml import etree
sns.set()


In [2]:
datamap=OrderedDict()
datamap["android"]="android.stackexchange.com"
datamap["dba"]="dba.stackexchange.com"
datamap["softwareEng"]="softwareengineering.stackexchange.com"
datamap["serverfault"]= "serverfault.com"
datamap["superuser"]="superuser.com"
datamap["stackoverflow"]="stackoverflow.com-"

In [3]:
def read_prog_lang(filename):
    data=OrderedDict()
    f=open(filename,"r")
    for l in f:
        row=l.split("\t")
        key=row[0].lower().strip()
        data[key]=[]
        for element in row[1].split(","):
            data[key].append(element.strip().lower())
    return data

def atleast_one(a, b):
    return not set(a).isdisjoint(b)

def refine_tags(tags):
    if tags!=None:
        l=tags.split('><')
        l=[i.replace('>','').replace('<','').lower() for i in l]
        return l
    else:
        return ["???"]

def init_data(logtags):
    data=OrderedDict()
    for lang in logtags:
        data[lang.lower()]=OrderedDict()
    return data

def read_tags(filename):
    logtags=[]
    for row in open(filename,"r"):
        logtags.append(row.split(",")[0].strip().lower())   
    return logtags

def check_tags(data,tags):
    for lang in langs:
        #print(tags,logtags[lang])
        if atleast_one(tags,prog_tags[lang]):
            # Atleast one logging tag
            return 1,lang

        elif atleast_one(tags,general_tags):
            if lang=="javascript":
                check=[lang,"js"]
            else:
                check=[lang]
            if atleast_one(tags,check):
                return 1, lang
    return 0,0

In [4]:
prog_tags=read_prog_lang("logging - Sheet4.tsv")
general_tags=list(set(read_tags("logging - Sheet3.csv")))
for i in prog_tags:
    general_tags.extend(prog_tags[i])
general_tags=list(set(general_tags))
langs=[i for i in prog_tags.keys()]

In [5]:
def distribute_posts(logtags):
    
    data=init_data(prog_tags)
    print(data)
    for website in list(datamap.keys())[5:6]:
        fname=website
        
        total_q,accepted_q,logging_q,total_logging_q=0,0,0,0
        
        if fname=="stackoverflow" and datamap[fname]=="stackoverflow.com-":
            datamap[fname]=datamap[fname]+"Posts"

        
        f=open("data/"+datamap[website]+"/Posts.xml","rb")
        next(f)
        next(f)
        for l in f:
            try:
                total_q+=1
                row=etree.XML(l)  
                posttypeid=row.get("PostTypeId") 
                acceptedanswerID=row.get("AcceptedAnswerId")
                tags=refine_tags(row.get("Tags"))
                if atleast_one(tags,general_tags):
                    total_logging_q+=1
                
                if posttypeid=="1" and acceptedanswerID!=None:
                    # Means it has an accepted answer
                    accepted_q+=1
                    
                    result,lang=check_tags(data,tags)
                    
                    if atleast_one(tags,general_tags):
                        logging_q+=1
                    
                    if result==1:
                        date=row.get("CreationDate")
                        key=date.split("T")[0][0:7]
                        if key not in data[lang]:
                            data[lang][key]=0
                        data[lang][key]+=1
                    
                    #print(tags)

            
            except:
                print(l)
        print(website,total_q,accepted_q,total_logging_q,logging_q)      
        f.close()
    return data

In [6]:
data=distribute_posts(prog_tags)
print(data)

OrderedDict([('java', OrderedDict()), ('c', OrderedDict()), ('c++', OrderedDict()), ('python', OrderedDict()), ('c#', OrderedDict()), ('javascript', OrderedDict()), ('android', OrderedDict())])


MemoryError: 